<img src="./Img/Banner_1.png">

8 ноября 2021 года<br>
ФРФТ20-2; вебинар, 11:50-13:20 <br>
ФРФТ20-1; вебинар, 14:00-15:30

<p style="text-align:center; font-size:150%">Контрольная работа</p>

<p style="font-size:110%">I. Изучите структуру базы данных магазина музыкальной продукции Чайнук.</p>

База данных Чайнук (Chinook) содержит сведения о работе музыкального Интернет-сервиса, аналогичного AppStore, в котором пользователи могут приобретать трэки, альбомы и составлять плейлисты. БД Chinook состоит из 11 таблиц:

- Таблица сотрудников хранит данные сотрудников, такие как идентификатор сотрудника, фамилия, имя и т. д. В ней есть поле с именем ReportsTo, чтобы указать, кто кому подчиняется. Сотрудники оказывают поддержку клиентам, для этого в таблице клиентов введен внешний ключ SupportRepId, ссылающийся на идентификатор сотрудника EmployeeId.
- В таблице клиентов хранятся данные о клиентах.
- Таблицы Invoice и InvoiceLine: в этих двух таблицах хранятся данные счетов. В таблице Invoice хранятся данные заголовка счета-фактуры, а в таблице InvoiceLine хранятся данные отдельных позиций счета-фактуры. Каждая строка InvoiceLine отражает информацию о заказе конкретного трека.
- В таблице исполнителей хранятся данные об артистах. Это простая таблица, содержащая только идентификатор и имя исполнителя.
- В таблице альбомов хранятся данные об альбоме и его исполнителе. Каждый альбом принадлежит одному исполнителю. Однако у одного исполнителя может быть несколько альбомов.
- В таблице MediaType хранятся типы мультимедиа, такие как аудиофайлы MPEG и аудиофайлы AAC.
- В таблице жанров хранятся типы музыки, такие как рок, джаз, металл и т. д.
- В таблице треков хранятся данные о песнях. Каждый трек принадлежит одному альбому.
- Таблицы Playlist и PlaylistTrack: таблица Playlist хранит данные о плейлистах. Каждый список воспроизведения содержит список треков. Каждый трек может принадлежать нескольким плейлистам. Связь между таблицей списков воспроизведения и таблицей треков - «многие ко многим». Таблица PlaylistTrack используется для отражения этой связи.

<img src="./Img/chinook_ERD.png" style="width:100%">

<p style="font-size:110%">II. Выполните отображение БД Chinook, создайте объекты таблиц.</p>

In [184]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, cast
from sqlalchemy import String, Numeric, Float, Integer
from sqlalchemy.sql import func, alias, desc

import pandas
from matplotlib import pyplot

engine = create_engine('sqlite:///Data/Chinook_Sqlite.sqlite')
metadata = MetaData()
metadata.reflect(bind=engine)
conn=engine.connect()

Album=Table("Album", metadata, autoload=True, autoload_with="engine")
Artist=Table("Artist", metadata, autoload=True, autoload_with="engine")
Customer=Table("Customer", metadata, autoload=True, autoload_with="engine")
Employee=Table("Employee", metadata, autoload=True, autoload_with="engine")
Track=Table("Track", metadata, autoload=True, autoload_with="engine")
Genre=Table("Genre", metadata, autoload=True, autoload_with="engine")
Invoice=Table("Invoice", metadata, autoload=True, autoload_with="engine")
InvoiceLine=Table("InvoiceLine", metadata, autoload=True, autoload_with="engine")
Playlist=Table("Playlist", metadata, autoload=True, autoload_with="engine")
PlaylistTrack=Table("PlaylistTrack", metadata, autoload=True, autoload_with="engine")

print(metadata.tables.keys())

dict_keys(['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'Track', 'MediaType', 'Playlist', 'PlaylistTrack'])


<p style="font-size:110%">III. Выполните задания в Jupyter notebook.</p>

1. Составьте запрос, выполняющий вывод количества клиентов (Customer) из каждой страны.
2. Составьте запрос, выполняющий вывод идентификаторов, имён, фамилий и стран клиентов (Customer) за исключением клиентов из США, Канады, Бразилии, Чили.
3. Составьте запрос, выполняющий вывод информации о счетах клиентов из Бразилии. Результат должен содержать фамилию и имя клиента, идентификатор счета, дату выставления счета, название страны плательщика (Billing Country).
4. Создайте запрос, показывающий, сколько счетов ассоциировано с каждым торговым агентом (Sales Support Agent). Результат должен содержать фамилию, имя агента, количество счетов.
5. Напишите запрос, выводящий на печать общее количество счетов, выставленных в 2009 и 2011 годах?
6. Составьте запрос, показывающий общее число трэков в каждом плей-листе. Результат должен содержать называние плей-листа.
7. Проанализируйте следующий код. Что является результатом? Скопируйте код в исполняемую ячейку, выполните и постройте столбчатую диаграмму.

```
E=Employee.alias("E")
I=Invoice.alias("I")
C=Customer.alias("C")

s=select(E.c.LastName + " " + E.c.FirstName, func.sum(cast(I.c.Total, Integer())))
s=s.where(E.c.EmployeeId==C.c.SupportRepId)
s=s.where(C.c.CustomerId==I.c.CustomerId)
s=s.where(E.c.Title.like("%Support%"))
s=s.group_by(E.c.LastName, E.c.FirstName)

result=conn.execute(s).fetchall()
df=pandas.DataFrame(result, columns=["Name","Value"])
df
```
8. Используя данные таблицы детализации счетов (InvoiceLine), постройте столбчатую диаграмму числа треков в заказах десяти самых популярных исполнителей.